In [1]:
import pandas as pd
import re

In [2]:
INPUT_CSV_FILE = './data/refined_data.csv' # From OpenRefine
OUTPUT_CSV_FILE = './data/cleaned_data.csv'

In [3]:
# @BEGIN clean_salary_data
# @IN input_csv_file @URI file:./data/refined_data.csv
# @OUT cleaned_data @URI file:./data/cleaned_data.csv

### Step 1: Reading input CSV

In [4]:
# @BEGIN read_csv_file
# @IN input_csv_file @URI file:./data/refined_data.csv
# @OUT raw_data
raw_data = pd.read_csv(INPUT_CSV_FILE)
raw_data
# @END read_csv_file

,Timestamp,Age,Industry,Job_Title,Salary,Currency,Location,Experience
0,4/24/2019 11:43:21,35-44,Government,Talent Management Asst. Director,75000,USD,"Nashville, TN, USA",11 - 20 years
1,4/24/2019 11:43:26,25-34,Environmental Consulting,Operations Director,65000,USD,"Madison, WI",8 - 10 years
2,4/24/2019 11:43:27,18-24,Market Research,Market Research Analyst,36330,USD,"Las Vegas, NV USA",2 - 4 years
3,4/24/2019 11:43:27,25-34,Biotechnology,Senior Scientist,34600,GBP,"Cardiff, UK",5-7 years
4,4/24/2019 11:43:29,25-34,Healthcare,Social worker (embedded in primary care),55000,USD,Southeast Michigan,5-7 years
...,...,...,...,...,...,...,...,...
34047,5/7/2020 16:19:27,35-44,Higher Ed,Department Head,125000,USD,"Chicago, IL",11 - 20 years
34048,5/9/2020 15:47:22,35-44,Government,Environmental Scientist,65000,USD,"St. Paul, MN, USA",11 - 20 years
34049,5/9/2020 20:35:05,35-44,Software,Senior Director,160000,USD,"Chicago, IL, USA",11 - 20 years
34050,5/10/2020 4:01:22,35-44,Newspapers,Reporter,39500,USD,"Los Angeles, CA, USA",11 - 20 years


### Step 2: Renaming data

In [5]:
# @BEGIN rename_columns
# @IN raw_data
# @OUT preprocessing_data @AS renamed_data
column_names = {"Salary": "Salary_Local"}
preprocessing_data = raw_data.rename(columns=column_names)
print("Data columns renamed:")
for key in column_names.keys():
    print("{} => {}".format(key, column_names[key]))
# @END rename_columns

Data columns renamed:
Salary => Salary_Local


### Step 3: Dropping records with NULL values

In [6]:
# @BEGIN drop_records_with_null_values
# @IN renamed_data
# @OUT raw_data @AS not_null_data
before_drops = len(preprocessing_data)
print("Number of records dropped for each column:")

for column in preprocessing_data.columns:
    before = len(preprocessing_data)
    preprocessing_data = preprocessing_data[preprocessing_data[column].notna()]
    n_dropped = before - len(preprocessing_data)
    print("{} - {}".format(column, n_dropped))

print("Total: {} records".format(before_drops - len(preprocessing_data)))
# @END drop_records_with_null_values

Number of records dropped for each column:
Timestamp - 0
Age - 0
Industry - 960
Job_Title - 0
Salary_Local - 11
Currency - 0
Location - 1357
Experience - 0
Total: 2328 records


### Step 4: Dropping records with invalid age based on experience

In [7]:
def parse_range(rnge):
    return [int(s) for s in re.split('\sor\s|\s-\s|-|\s', rnge) if s.isdigit()]

# @BEGIN extract_floor_age
# @IN not_null_data
# @OUT not_null_data @AS not_null_data_with_floor_age
preprocessing_data['floor_age'] = preprocessing_data['Age'].apply(lambda age_range: parse_range(age_range)[0])
# @END extract_floor_age

# @BEGIN extract_floor_experience
# @IN not_null_data_with_floor_age
# @OUT not_null_data @AS not_null_data_with_floor_age_and_experience
preprocessing_data['floor_experience'] = preprocessing_data['Experience'].apply(lambda exp_range: parse_range(exp_range)[0])
# @END extract_floor_experience

# @BEGIN filter_out_invalid_records_based_on_age_and_experience
# @IN not_null_data_with_floor_age_and_experience
# @OUT filtered_data
filtered_data = preprocessing_data[preprocessing_data["floor_age"] > preprocessing_data["floor_experience"]]
# @END filter_out_invalid_records_based_on_age_and_experience

print("BEFORE removing invalid Age: {} records".format(len(preprocessing_data)))
print("AFTER removing invalid Age: {} records".format(len(filtered_data)))

BEFORE removing invalid Age: 31724 records
AFTER removing invalid Age: 31713 records


### Step 5: Removing undetermined currencies

In [8]:
# @BEGIN remove_records_with_undetermined_currency
# @IN filtered_data
# @OUT filtered_data @AS filtered_data_without_undetermined_currency
print("BEFORE removing undetermined currency: {} records".format(len(filtered_data)))
filtered_data = filtered_data[filtered_data["Currency"] != "Other"]
print("AFTER removing undetermined currency: {} records".format(len(filtered_data)))
# @END remove_records_with_undetermined_currency

BEFORE removing undetermined currency: 31713 records
AFTER removing undetermined currency: 31539 records


### Step 6: Converting salaries to USD

In [9]:
# @BEGIN convert_salaries_to_usd
# @PARAM usd_conversion_rates
# @IN filtered_data_without_undetermined_currency
# @OUT filtered_data @AS usd_salaries_data
usd_conversion_rates = {
    "USD": 1,
    "GBP": 1.37,
    "CAD": 0.80,
    "EUR": 1.18,
    "SEK": 0.12,
    "AUD/NZD": 0.74,
    "JPY": 0.0091,
    "CHF": 1.09,
    "HKD": 0.13,
    "ZAR": 0.067
}
print("View currencies:")
print(filtered_data["Currency"].unique())
filtered_data["Salary_USD"] = filtered_data.apply(lambda row: usd_conversion_rates[row["Currency"]] * float(row["Salary_Local"]), axis=1)
# @END convert_salaries_to_usd

View currencies:
['USD' 'GBP' 'CAD' 'EUR' 'SEK' 'AUD/NZD' 'JPY' 'CHF' 'HKD' 'ZAR']


### Step 7: Exporting Cleaned Data

In [10]:
# @BEGIN export_clean_data
# @IN usd_salaries_data
# @OUT cleaned_data @URI file:./data/cleaned_data.csv
cleaned_data = filtered_data[[
    "Timestamp",
    "Age",
    "Location",
    "Industry",
    "Job_Title",
    "Experience",
    "Salary_USD",
    "Salary_Local",
    "Currency"
]]

cleaned_data.to_csv(OUTPUT_CSV_FILE, index=False)
cleaned_data
# @END convert_salaries_to_usd

,Timestamp,Age,Location,Industry,Job_Title,Experience,Salary_USD,Salary_Local,Currency
0,4/24/2019 11:43:21,35-44,"Nashville, TN, USA",Government,Talent Management Asst. Director,11 - 20 years,75000.0,75000,USD
1,4/24/2019 11:43:26,25-34,"Madison, WI",Environmental Consulting,Operations Director,8 - 10 years,65000.0,65000,USD
2,4/24/2019 11:43:27,18-24,"Las Vegas, NV USA",Market Research,Market Research Analyst,2 - 4 years,36330.0,36330,USD
3,4/24/2019 11:43:27,25-34,"Cardiff, UK",Biotechnology,Senior Scientist,5-7 years,47402.0,34600,GBP
4,4/24/2019 11:43:29,25-34,Southeast Michigan,Healthcare,Social worker (embedded in primary care),5-7 years,55000.0,55000,USD
...,...,...,...,...,...,...,...,...,...
34047,5/7/2020 16:19:27,35-44,"Chicago, IL",Higher Ed,Department Head,11 - 20 years,125000.0,125000,USD
34048,5/9/2020 15:47:22,35-44,"St. Paul, MN, USA",Government,Environmental Scientist,11 - 20 years,65000.0,65000,USD
34049,5/9/2020 20:35:05,35-44,"Chicago, IL, USA",Software,Senior Director,11 - 20 years,160000.0,160000,USD
34050,5/10/2020 4:01:22,35-44,"Los Angeles, CA, USA",Newspapers,Reporter,11 - 20 years,39500.0,39500,USD


In [11]:
print("\n[DONE] Data exported to {}".format(OUTPUT_CSV_FILE))
# @END clean_salary_data


[DONE] Data exported to ./data/cleaned_data.csv
